In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

#model validation
from sklearn.metrics import log_loss,roc_auc_score,precision_score,f1_score,recall_score,roc_curve,auc
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,fbeta_score,matthews_corrcoef
from sklearn import metrics

# cross validation
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_csv("/content/drive/MyDrive/liver cirrhosis/Liver cirrhosis UCI Dataset.csv")
df.head()

,AGE,GENDER,TB(Total Bilirubin),DB(Direct Bilirubin),Alkphos Alkaline Phosphotase,Sgpt Alamine Aminotransferase,Sgot Aspartate Aminotransferase,TP Total Protiens,ALB Albumin,A/G Ratio Albumin and Globulin Ratio,Selector field used to split the data into two sets (labeled by the experts)
0,4,Male,0.9,0.2,348,30,34,8.0,4.0,1.0,2
1,4,Male,0.8,0.2,460,152,231,6.5,3.2,0.9,2
2,6,Male,0.6,0.1,289,38,30,4.8,2.0,0.7,2
3,7,Female,27.2,11.8,1420,790,1050,6.1,2.0,0.4,1
4,7,Male,0.5,0.1,352,28,51,7.9,4.2,1.1,2


In [ ]:
df.dtypes

AGE                                                                               int64
GENDER                                                                           object
TB(Total Bilirubin)                                                             float64
DB(Direct Bilirubin)                                                            float64
Alkphos Alkaline Phosphotase                                                      int64
Sgpt  Alamine Aminotransferase                                                    int64
Sgot Aspartate Aminotransferase                                                   int64
TP Total Protiens                                                               float64
ALB Albumin                                                                     float64
A/G Ratio Albumin and Globulin Ratio                                            float64
Selector field used to split the data into two sets (labeled by the experts)      int64
dtype: object

In [ ]:
newdf=df.dropna(axis=0, how="any")

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

X = newdf.drop(['Selector field used to split the data into two sets (labeled by the experts)'],axis=1)


# Perform label encoding for categorical variables (if any)
categorical_cols = ['GENDER']  # Replace with the names of your categorical columns
label_encoder = LabelEncoder()
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd

# Assuming you have your dataset stored in a Pandas DataFrame called "df" with features in X and the classification column in y

# Extract the classification column
classification = newdf['Selector field used to split the data into two sets (labeled by the experts)']

# Apply label encoding to convert "yes" and "no" to numerical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(classification)

# Scale the features using Min-Max scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Convert the encoded classification column to probability values of "yes" in percentage
class_probabilities = y_encoded / (y_encoded.max() - y_encoded.min())

# Create a new DataFrame with scaled features and probability values
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['Probability_of_Yes'] = class_probabilities

# Print the updated scaled dataset
print(df_scaled)

          AGE  GENDER  TB(Total Bilirubin)  DB(Direct Bilirubin)  \
0    0.000000     1.0             0.006702              0.005102   
1    0.000000     1.0             0.005362              0.005102   
2    0.023256     1.0             0.002681              0.000000   
3    0.034884     0.0             0.359249              0.596939   
4    0.034884     1.0             0.001340              0.000000   
..        ...     ...                  ...                   ...   
574  0.825581     1.0             0.006702              0.005102   
575  0.860465     1.0             0.008043              0.010204   
576  0.930233     0.0             0.004021              0.005102   
577  0.941860     0.0             0.008043              0.010204   
578  1.000000     1.0             0.009383              0.010204   

     Alkphos Alkaline Phosphotase  Sgpt  Alamine Aminotransferase  \
0                        0.139228                        0.010050   
1                        0.193942            

In [ ]:
y=df_scaled['Probability_of_Yes']
X = df_scaled.drop(['Probability_of_Yes'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
# Assuming you have your dataset stored in a Pandas DataFrame called "df" with the features in X and the target variable in y

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Apply polynomial features
poly = PolynomialFeatures(degree=1)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
# Define the preprocessing pipeline
preprocessing_pipeline = make_pipeline(
    PolynomialFeatures(degree=1),
   StandardScaler())
# Apply preprocessing pipeline to training and testing sets
X_train_preprocessed = preprocessing_pipeline.fit_transform(X_train_poly)
X_test_preprocessed = preprocessing_pipeline.transform(X_test_poly)

# Create and train the logistic regression model
model = LogisticRegression(C=10,max_iter=10000,solver='liblinear')
model.fit(X_train_preprocessed, y_train)
#No use of applying feature selection and SMOTE
# Predict the target variable on the testing set
y_pred = model.predict(X_test_preprocessed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7758620689655172


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Assuming you have your dataset stored in a Pandas DataFrame called "df" with the features in X and the target variable in y

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Apply polynomial features and feature scaling
poly = PolynomialFeatures(degree=1)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# Define the logistic regression model
model = LogisticRegression(C=1,max_iter=100000,solver='liblinear')

# Define additional models for ensemble stacking
svm_model = SVC(C=100,kernel='rbf')
gb_model = GradientBoostingClassifier(max_depth=2,min_samples_split=4,min_samples_leaf=1,max_features='sqrt')
dt_model = DecisionTreeClassifier(max_depth=2,min_samples_split=4,min_samples_leaf=1)

# Define the stacking classifier
stacking_model = StackingClassifier(
    estimators=[('logistic', model), ('svm', svm_model), ('gb', gb_model), ('dt', dt_model)],
    final_estimator=model
)

# Fit the stacking classifier on the training data
stacking_model.fit(X_train_scaled, y_train)

# Predict the target variable on the testing set
y_pred = stacking_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7873563218390804


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Define a function to get user input and make predictions
def predict_cirrhosis_probability():
 #defining the features of the dataset
    age = float(input("Enter age: "))
    gender = int(input("Enter gender (0 for female, 1 for male): "))
    total_bilirubin = float(input("Enter Total Bilirubin: "))
    direct_bilirubin = float(input("Enter Direct Bilirubin: "))
    alkaline_phosphotase = int(input("Enter Alkphos Alkaline Phosphotase: "))
    alamine_aminotransferase = int(input("Enter Sgpt Alamine Aminotransferase: "))
    aspartate_aminotransferase = int(input("Enter Sgot Aspartate Aminotransferase: "))
    total_proteins = float(input("Enter TP Total Protiens: "))
    albumin = float(input("Enter ALB Albumin: "))
    albumin_globulin_ratio = float(input("Enter A/G Ratio Albumin and Globulin Ratio: "))

    # Preprocess the user input as done during training
    user_input = np.array([[age, gender, total_bilirubin, direct_bilirubin, alkaline_phosphotase,
                            alamine_aminotransferase, aspartate_aminotransferase, total_proteins,
                            albumin, albumin_globulin_ratio]])
        #['AGE','GENDER','TB(Total Bilirubin)','DB(Direct Bilirubin)','Alkphos Alkaline Phosphotase','Sgpt  Alamine Aminotransferase','Sgot Aspartate Aminotransferase','TP Total Protiens','ALB Albumin','A/G Ratio Albumin and Globulin Ratio'])
    user_input_poly = poly.transform(user_input)
    user_input_scaled = scaler.transform(user_input_poly)

    # Use the trained stacking model to make predictions1
    probability = stacking_model.predict_proba(user_input_scaled)[0][1]
    return probability

# Call the function to get the predicted probability
predicted_probability = predict_cirrhosis_probability()
print("Predicted Probability of Liver Cirrhosis:", predicted_probability)
print("If the value of predicted probability is less than 0.1204 then liver cirrhosis doesnt exists otherwise it does")

Enter age: 4
Enter gender (0 for female, 1 for male): 1
Enter Total Bilirubin: 0.8
Enter Direct Bilirubin: 0.2
Enter Alkphos Alkaline Phosphotase: 460
Enter Sgpt Alamine Aminotransferase: 152
Enter Sgot Aspartate Aminotransferase: 231
Enter TP Total Protiens: 6.5
Enter ALB Albumin: 3.2
Enter A/G Ratio Albumin and Globulin Ratio: 0.9
Predicted Probability of Liver Cirrhosis: 0.12023166633128667
If the value of predicted probability is less than 0.1204 then liver cirrhosis exists otherwise it doesn't


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


In [ ]:
134